<a href="https://colab.research.google.com/github/KhawajaAbaid/kaggle-playground/blob/main/PS03E02_Tuning_CatBoost_with_feat_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"khawajaabaidullah","key":"2086e1f7ef36dba18df19ebd6485e99d"}'}

In [2]:
!mkdir ~/.kaggle
!cp ./kaggle.json ~/.kaggle/kaggle.json

In [3]:
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c playground-series-s3e2

100% 321k/321k [00:00<00:00, 716kB/s]
100% 321k/321k [00:00<00:00, 715kB/s]


In [5]:
!unzip ./playground-series-s3e2.zip

Archive:  ./playground-series-s3e2.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
!pip install optuna

In [18]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 7.9 MB/s eta 0:00:00


In [19]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import lightgbm as lgbm
import catboost
import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import optuna

In [8]:
from warnings import filterwarnings
filterwarnings('ignore')

In [9]:
train = pd.read_csv("train.csv").drop(columns="id")
test = pd.read_csv("test.csv").drop(columns="id")

In [10]:
df = pd.concat([train.drop(columns=["stroke"]), test], axis=0).reset_index(drop=True)
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked
1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked
2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown
3,Male,56.0,0,0,Yes,Private,Urban,64.87,28.8,never smoked
4,Female,24.0,0,0,No,Private,Rural,73.36,28.8,never smoked


# Feature Engineering

In [11]:
df['morbid'] = np.where(df.bmi>40,1,0)
df['obese'] = np.where(df.bmi>30,1,0)
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,morbid,obese
0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked,0,1
1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked,0,0
2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown,1,1
3,Male,56.0,0,0,Yes,Private,Urban,64.87,28.8,never smoked,0,0
4,Female,24.0,0,0,No,Private,Rural,73.36,28.8,never smoked,0,0


In [12]:
def feature_risk_factors(df):
    df["risk_factors"] = df[[
        "avg_glucose_level", "age", "bmi", 
        "hypertension", "heart_disease", 
        "smoking_status"
    ]].apply(
        lambda x: \
        0 + (1 if x.avg_glucose_level > 99 else 0) + \
        (1 if x.age > 45 else 0) + (1 if x.bmi > 24.99 else 0) + \
        (1 if x.hypertension == 1 else 0) + \
        (1 if x.heart_disease == 1 else 0) + \
        (1 if x.smoking_status in ["formerly smoked", "smokes"] else 0),
        axis=1
    )
    return df

In [ ]:
feature_risk_factors(df)

In [14]:
df = pd.get_dummies(df)
df.head()

,age,hypertension,heart_disease,avg_glucose_level,bmi,morbid,obese,risk_factors,gender_Female,gender_Male,...,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,28.0,0,0,79.53,31.1,0,1,1,0,1,...,0,1,0,0,0,1,0,0,1,0
1,33.0,0,0,78.44,23.9,0,0,1,0,1,...,0,1,0,0,1,0,0,1,0,0
2,42.0,0,0,103.00,40.3,1,1,2,1,0,...,0,1,0,0,1,0,1,0,0,0
3,56.0,0,0,64.87,28.8,0,0,2,0,1,...,0,1,0,0,0,1,0,0,1,0
4,24.0,0,0,73.36,28.8,0,0,1,1,0,...,0,1,0,0,1,0,0,0,1,0


In [15]:
X = df.iloc[:-len(test), :]
test_new = df.iloc[-len(test):, :]
y = train.stroke

In [20]:
def objective(trial, X, y):
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 500, 3000, step=100),
        "loss_function": trial.suggest_categorical("loss_function", ["CrossEntropy"]),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 1e0),
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-2, 1e0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 10),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 2, 20),
        "one_hot_max_size": trial.suggest_int("one_hot_max_size", 2, 20),  
    }
    # Conditional Hyper-Parameters
    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)
    
    cv = StratifiedKFold(n_splits=8, shuffle=True, random_state=1337)

    cv_scores = np.empty(8)
    for fold, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        cat_model = catboost.CatBoostClassifier(**param)
        cat_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=0, early_stopping_rounds=100)
        
        y_preds = cat_model.predict_proba(X_test)[:, 1]
        cv_scores[fold] = roc_auc_score(y_test, y_preds)
    
    auc = np.mean(cv_scores)
    print(f"AVG CV AUC: \t {auc}")
    return auc

In [21]:
study = optuna.create_study(direction="maximize", study_name="CatBoost Tuning")
func = lambda trial: objective(trial, X, y)
study.optimize(func, n_trials=100, show_progress_bar=True)

[I 2023-01-16 09:04:22,635] A new study created in memory with name: CatBoost Tuning


  0%|          | 0/100 [00:00<?, ?it/s]

AVG CV AUC: 	 0.8824248029485251
[I 2023-01-16 09:05:51,793] Trial 0 finished with value: 0.8824248029485251 and parameters: {'n_estimators': 900, 'loss_function': 'CrossEntropy', 'learning_rate': 0.0363464276658744, 'l2_leaf_reg': 0.3661925904464463, 'colsample_bylevel': 0.07724807323558153, 'depth': 5, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli', 'min_data_in_leaf': 20, 'one_hot_max_size': 20, 'subsample': 0.5161878537924918}. Best is trial 0 with value: 0.8824248029485251.
AVG CV AUC: 	 0.8513378621813011
[I 2023-01-16 09:06:35,435] Trial 1 finished with value: 0.8513378621813011 and parameters: {'n_estimators': 1700, 'loss_function': 'CrossEntropy', 'learning_rate': 0.001564925784305862, 'l2_leaf_reg': 0.40917269608936346, 'colsample_bylevel': 0.034300146350872945, 'depth': 2, 'boosting_type': 'Plain', 'bootstrap_type': 'MVS', 'min_data_in_leaf': 2, 'one_hot_max_size': 10}. Best is trial 0 with value: 0.8824248029485251.
AVG CV AUC: 	 0.849642132435156
[I 2023-01-16 0

In [22]:
study.best_value

0.8883958767582789

In [23]:
study.best_params

{'n_estimators': 1300,
 'loss_function': 'CrossEntropy',
 'learning_rate': 0.40313035898789806,
 'l2_leaf_reg': 0.024122458230293627,
 'colsample_bylevel': 0.05829673202440563,
 'depth': 1,
 'boosting_type': 'Plain',
 'bootstrap_type': 'MVS',
 'min_data_in_leaf': 20,
 'one_hot_max_size': 12}